# Load the model

In [1]:
!pip install -qqq transformers==4.44.2 accelerate==0.33.0 mistral_inference langchain youtube_transcript_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 717.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━

## Text Generation Model

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [73]:
def generate_text(prompt, max_length=100, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Downlaod the Transcript

In [74]:
from youtube_transcript_api import YouTubeTranscriptApi

def extract_video_id(url):
    """Extract the video ID from the YouTube URL."""
    if 'youtube.com/watch?v=' in url:
        return url.split('v=')[1].split('&')[0]
    elif 'youtu.be/' in url:
        return url.split('youtu.be/')[1].split('?')[0]
    else:
        raise ValueError("Invalid YouTube URL")

def fetch_transcript(video_id):
    """Fetch the transcript of the YouTube video."""
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

def print_transcript(transcript):
    """Print the transcript text."""
    full_text = ''
    for entry in transcript:
        full_text += entry['text']
    return full_text

In [75]:
video_url = 'LINK_YouTube'
video_id = extract_video_id(video_url)
transcript = fetch_transcript(video_id)

In [76]:
final_transcript = print_transcript(transcript)

In [ ]:
len(final_transcript)

# Chunking

In [93]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = " ",
    chunk_size = 2000,
    chunk_overlap = 0,
    length_function = len
)

In [94]:
normal_chunks = text_splitter.split_text(
    final_transcript
    )

In [95]:
question_num = int(10/ len(normal_chunks))

In [96]:
print("Number of chunks: ",len(normal_chunks), "Chunks")
print("every question has: ",question_num, "Questions")

Number of chunks:  9 Chunks
every question has:  1 Questions


In [97]:
from langchain.prompts import PromptTemplate
import re
import random
import time

In [98]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)

    return f"```json\n{matches[2]}\n```"

In [99]:
if question_num > 1:
  ques_num = f"{question_num} random questions"
else:
  ques_num = "One random question"

ques_num

'One random question'

In [100]:
if question_num > 1:
  ans_num = f"{question_num} true short answers"
else:
  ans_num = "one true short answer"
ans_num

'one true short answer'

# LangChain Parser

In [101]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [102]:
ques_schema = ResponseSchema(name="questions",
                             description=f"A list of {ques_num} about the text.")

true_ans_schema = ResponseSchema(name="true_answers",
                                      description=f"A list of {ans_num} for each question from the text, corresponding to the same index in the questions list.")

false_ans_schema = ResponseSchema(name="false_answers",
                                      description="A list of lists, where each sublist contains 3 false answers for the corresponding question in the questions list.\
                                                   Each false answer should resemble the true answer. Output these as a proper Python list of lists (e.g., [['answer1', 'answer2', 'answer3'], ['answer4', 'answer5', 'answer6']]).")

response_schemas = [ques_schema,
                    true_ans_schema,
                    false_ans_schema]

In [103]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [104]:
format_instructions = output_parser.get_format_instructions()

In [105]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"questions": string  // A list of One random question about the text.
	"true_answers": string  // A list of one true short answer for each question from the text, corresponding to the same index in the questions list.
	"false_answers": string  // A list of lists, where each sublist contains 3 false answers for the corresponding question in the questions list.                                                   Each false answer should resemble the true answer. Output these as a proper Python list of lists (e.g., [['answer1', 'answer2', 'answer3'], ['answer4', 'answer5', 'answer6']]).
}
```


# Start Code

In [107]:
# time
t = time.time()
for j in range(len(normal_chunks)):
    print("Chunck: ", j+1)

    review_template = """\
    For the following text, extract the following information:

    questions: A list of {ques_num} about the text.

    true_answers: A list of {ans_num} for each question from the text, corresponding to the same index in the questions list.

    false_answers: A list of lists, where each sublist contains 3 false answers for the corresponding question in the questions list.\
                  Each false answer should resemble the true answer. Output these as a proper Python list of lists (e.g., [['answer1', 'answer2', 'answer3'], ['answer4', 'answer5', 'answer6']]).

    text: {text}

    {format_instructions}
    """


    prompt = PromptTemplate(template=review_template)

    messages = prompt.format(text=normal_chunks[j],
                             format_instructions=format_instructions,
                             ques_num = ques_num,
                             ans_num = ans_num)


    response = generate_text(messages, max_length=1500 + (100 * question_num), num_return_sequences=1)

    try:
      final_response = extract_json_block(response[0])
    except:
      pass
      # print("error - 1")
      # print(response[0])

    try:
      # Check the parser
      output_dict = output_parser.parse(final_response)

      # Append True answer with wrong answers
      for i, answer in enumerate(output_dict['true_answers']):
        ans = answer
        output_dict["false_answers"][i].append(ans)

      # print the Questions
      for i, question in enumerate(output_dict['questions']):

        question_text = question
        choices = output_dict["false_answers"][i]
        random.shuffle(choices)

        correct_answer = output_dict["true_answers"][i]

        print(f"""
        Question:
        {question_text}

        Choices:
        a) {choices[0]}
        b) {choices[1]}
        c) {choices[2]}
        d) {choices[3]}

        Answer: {correct_answer}
        """)
        print("---------------------------")


    except:
      pass
        # print("erorr - 2")
        # print(final_response)

    print("------------------------------------")

print("Turn number:", j+1, " take: ", time.time() - t)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Chunck:  1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        Question:
        What is the expected AI market size by the year 2030?

        Choices:
        a) 20 volt
        b) 15 volt
        c) 20 volt
        d) 25 volt

        Answer: 20 volt
        
---------------------------
------------------------------------
Chunck:  2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


        Question:
        What is the best way to learn Python according to the text?

        Choices:
        a) By taking online courses
        b) By attending workshops
        c) By downloading and reverse-engineering projects from others.
        d) By reading books

        Answer: By downloading and reverse-engineering projects from others.
        
---------------------------
------------------------------------
Chunck:  7


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        Question:
        What is the main goal of the project?

        Choices:
        a) To offer a platform for social networking.
        b) To provide a guide on gardening.
        c) To provide a learning resource for data science and AI.
        d) To teach cooking recipes.

        Answer: To provide a learning resource for data science and AI.
        
---------------------------
------------------------------------
Chunck:  8


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


------------------------------------
Chunck:  9
------------------------------------
Turn number: 9  take:  822.6962399482727
